In [2]:
import os
import pandas as pd
import numpy as np
import awkward
import uproot_methods
print(ak.__version__)

0.12.0


In [19]:
def SetAKArr(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    px_lists = []
    px_ls = []
    for line in lines:
        if line.startswith('E'):
            px_lists.append(px_ls)
            ##clear the data in px list
#             print(px_ls)
            px_ls = []
        else:
            par = line.split()
            px_ls.append(par[1])

In [20]:
SetAKArr("train.txt")

[[], ['-0.3162697467459443', '0.02881739128528447', '0.21155536277901646', '0.1127395856295861', '0.04950450296327485', '0.05790811731263832', '-0.11239735132152338', '0.33168517010613346', '-1.0925161813823512', '-0.05187720252769525', '0.027687138267806676', '0.2178420363815063', '0.09674890900651571', '0.050166316629249634', '-0.03653761742925005', '0.03266150952930407', '0.12861184258278482', '0.08675066284909895', '0.24405372120757218'], ['0.40624541033679146', '-0.026873773856092374', '-0.24371108140810097', '0.06375648800626758', '-0.035557062314887096', '-0.09613518565411182', '-0.04482796590902382', '-0.12137819267594728', '-0.04527494337494476', '-0.04658473864684581', '-0.15217238940014025', '-0.05181183921518359', '0.2970857281216646', '0.4257168039675718', '-0.13120436464506557', '-0.025704445225620598', '-0.07454940257199988', '0.030960933939939032', '0.02788891197534895'], ['-0.20377421531128534', '0.0861862008640884', '-0.01919939482246239', '-0.16422959435684992', '0.1

In [3]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
print(myarray[1:])
# returns <JaggedArray [[] [4.4 5.5]] at 7f02018afc18>
print(myarray[100:])
# returns <JaggedArray [] at 7f020c214438>

[[] [4.4 5.5]]
[]


In [4]:
myarray = ak.JaggedArray.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
indexes = ak.JaggedArray.fromiter([[1, 2, 2, 0], [], [1]])
myarray[indexes]
# returns <JaggedArray [[3.3 3.3 1.1] [] [5.5]] at 7f02018afa58>

<JaggedArray [[2.2 3.3 3.3 1.1] [] [5.5]] at 0x7f1b71f976a0>

In [14]:
myarray = ak.JaggedArray.fromcounts([3, 0, 2], ak.Table(
              x=[1, 2, 3, 4, 5],
              y=[1.1, 2.2, 3.3, 4.4, 5.5],
              z=[True, False, True, False, False]))
print(myarray["x"])
# returns <JaggedArray [[1 2 3] [] [4 5]] at 7f020e8122b0>
print(myarray[["x", "y"]])
# returns <JaggedArray [[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]] at 7f02018af860>
print(myarray[["x", "y"]].columns)
# returns ['x', 'y']
print(myarray.count())
print(myarray)
myarray["x"] = [1, 2, 3, 4, 5]
print(myarray["x"])

[[1 2 3] [] [4 5]]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
['x', 'y']
[<Row 0> <Row 1> <Row 2>]
[[<Row 0> <Row 1> <Row 2>] [] [<Row 3> <Row 4>]]
[[1 1 1] [] [3 3]]
